In [116]:
import matplotlib.pyplot as plt
import numpy as np
import scipy

# Grid Info

In [117]:
NX = 11
NY = 11
LX = 1
LY = 1
dx = LX / (NX - 1)
dy = LY / (NY - 1)
dt = 0.01
X = np.linspace(0, LX, NX)
Y = np.linspace(0, LY, NY)
XMG, YMG = np.meshgrid(X, Y, indexing="ij")
X, Y, dx, dy

(array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
 array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
 0.1,
 0.1)

# Fluid Properties

In [118]:
mu = 0.1
rho = 1.0

lid_velocity = 2.0

# Convert from Field to Vector and Back

In [119]:
def convert_field_to_vector(field):
    """Flattens a field to a vector. Iteration is done row first.

    :param field: shape (i,j)
    :return vector: shape (i*j)
    """
    nx, ny = field.shape
    vector = np.zeros(nx * ny)
    for jdx in range(ny):
        for idx in range(nx):
            ij = jdx * nx + idx
            vector[ij] = field[idx, jdx]
    return vector

def convert_vector_to_field(vector, nx, ny):
    """Stacks a vector into a field. Iteration is done row first.

    :param vector: shape (i*j)
    :return field: shape (i,j)
    """
    field = np.zeros((nx, ny))
    for jdx in range(ny):
        for idx in range(nx):
            ij = jdx * nx + idx
            field[idx, jdx] = vector[ij]
    return field

vec = np.arange(0,20,1)
print(vec, vec.shape)

fld = convert_vector_to_field(vec, 5, 4)
print(fld, fld.shape)

back2vec = convert_field_to_vector(fld)
print(back2vec, back2vec.shape)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] (20,)
[[ 0.  5. 10. 15.]
 [ 1.  6. 11. 16.]
 [ 2.  7. 12. 17.]
 [ 3.  8. 13. 18.]
 [ 4.  9. 14. 19.]] (5, 4)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19.] (20,)


# Projection Method

From the [Wikipedia page on the projection method:](https://en.wikipedia.org/wiki/Projection_method_(fluid_dynamics))

The incompressible Navier-Stokes equation is (in vector form):
\begin{equation} \tag{NSE}
\frac{\partial \mathbf{u}}{\partial t} + (\mathbf{u} \cdot \nabla) \mathbf{u} = - \frac{1}{\rho} \nabla p + \nu \nabla^2 \mathbf{u}
\end{equation}

where $\nu = \mu / \rho$: the kinematic viscosity is equal to the dynamic viscosity divided by the density.

The two steps of the projection method are as follows:

## 1) Compute an Intermediate Velocity

\begin{equation} \tag{1}
\frac{\mathbf{u}^* - \mathbf{u}^n}{\Delta t} = - (\mathbf{u}^n \cdot \nabla) \mathbf{u}^n + \nu \nabla^2 \mathbf{u}^n
\end{equation}

Note that this equation presents an explicit step. Implicit and semi-implicit steps are also possible.

## 2) Correct the Intermediate Velocity

\begin{equation} \tag{2}
\mathbf{u}^{n+1} = \mathbf{u}^* - \frac{\Delta t}{\rho} \nabla p^{n+1}
\end{equation}

This step can be rewritten as follows, to make clear that the algorithm is fundamentally an operator splitting approach. The first step only considers viscous forces, while the second step considers the pressure forces. The result from the first step is "projected" back into the space of incompressibilty.
\begin{equation} \nonumber{}
\frac{\mathbf{u}^{n+1} - \mathbf{u}^*}{\Delta t} = - \frac{1}{\rho} \nabla p^{n+1}
\end{equation}

The right hand side of Step 2 requires knowledge of the pressure field. This can be obtained by solving the following Poisson equation:
\begin{equation} \tag{1.5}
\nabla^2 p^{n+1} = \frac{\rho}{\Delta t} \nabla \cdot \mathbf{u}^*
\end{equation}

# Numerical Experiments

Consider the lid-driven cavity on the grid described above. A velocity is prescribed of 0.1 m/s to the right at the lid. Let $\Omega$ represent the edge of the domain.

\begin{equation} \notag{}
\begin{split}
v = 0 \space \forall \space v \in \Omega \\
u(x=0) = 0\\
u(y=0) = 0 \\
u(y=LY) = 0.1 \\
u(x=LX) = 0 \\
\end{split}
\end{equation}



## Explicit Step to Intermediate Velocity

\begin{equation} \notag{}
\frac{\mathbf{u}^* - \mathbf{u}^n}{\Delta t} = - (\mathbf{u}^n \cdot \nabla) \mathbf{u}^n + \nu \nabla^2 \mathbf{u}^n
\end{equation}

Expanding into two dimensions:
\begin{split}
u^* = u^n + \Delta t \left( - u\frac{\partial u}{\partial x} - v\frac{\partial u}{\partial y} + \frac{\mu}{\rho} \left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right) \right) \\
v^* = v^n + \Delta t \left( - u\frac{\partial v}{\partial x} - v\frac{\partial v}{\partial y} + \frac{\mu}{\rho} \left( \frac{\partial^2 v}{\partial x^2} + \frac{\partial^2 v}{\partial y^2} \right) \right) \\
\end{split}

Discretizing:
\begin{split}

\end{split}

In [129]:
u_n = np.zeros(NX * NY)
v_n = np.zeros(NX * NY)

u_star = np.zeros(NX * NY)
v_star = np.zeros(NX * NY)

# set u b.c. at top of cavity
for jdx in range(NY):
    for idx in range(NX):
        ij = jdx * NX + idx
        if jdx == NY - 1:
            u_n[ij] = lid_velocity

# calculate intermediate velocity
for jdx in range(NY):
    for idx in range(NX):
        ij = jdx * NX + idx
        im1 = jdx * NY + idx - 1
        ip1 = jdx * NY + idx + 1
        jm1 = (jdx - 1) * NY + idx
        jp1 = (jdx + 1) * NY + idx
        # set boundary conditions
        if jdx == NY - 1:
            u_star[ij] = lid_velocity
            v_star[ij] = 0
            continue
        elif jdx == 0 or idx == 0 or idx == NX - 1:
            u_star[ij] = 0
            v_star[ij] = 0
            continue
        # grid steps
        dx_p = X[idx + 1] - X[idx]
        dx_m = X[idx] - X[idx - 1]
        dy_p = Y[idx + 1] - Y[idx]
        dy_m = Y[idx] - Y[idx - 1]
        # first order derivatives
        dudx_p = (u_n[ip1] - u_n[ij]) / dx_p
        dudx_m = (u_n[ij] - u_n[im1]) / dx_m
        dudy_p = (u_n[jp1] - u_n[ij]) / dy_p
        dudy_m = (u_n[ij] - u_n[jm1]) / dy_m
        dvdx_p = (v_n[ip1] - v_n[ij]) / dx_p
        dvdx_m = (v_n[ij] - v_n[im1]) / dx_m
        dvdy_p = (v_n[jp1] - v_n[ij]) / dy_p
        dvdy_m = (v_n[ij] - v_n[jm1]) / dy_m
        # first order terms
        ududx = u_n[ij] * 0.5 * (dudx_p + dudx_m)
        vdudy = v_n[ij] * 0.5 * (dudy_p + dudy_m)
        udvdx = u_n[ij] * 0.5 * (dvdx_p + dvdx_m)
        vdvdy = v_n[ij] * 0.5 * (dvdy_p + dvdy_m)
        # second order terms
        d2udx2 = (dudx_p - dudx_m) / (0.5 * (dx_p + dx_m))
        d2udy2 = (dudy_p - dudy_m) / (0.5 * (dy_p + dy_m))
        d2vdx2 = (dvdx_p - dvdx_m) / (0.5 * (dx_p + dx_m))
        d2vdy2 = (dvdy_p - dvdy_m) / (0.5 * (dy_p + dy_m))
        # store intermediate velocities
        u_star[ij] = u_n[ij] + dt * (-ududx - vdudy + (mu/rho) * (d2udx2 + d2udy2))
        v_star[ij] = v_n[ij] + dt * (-udvdx - vdvdy + (mu/rho) * (d2vdx2 + d2vdy2))

In [ ]:
"""20251105@11:51PM
The output of the above cell is confirmed to match the chorin projection tentative velocity from the notebook in the 2d heat conduction repo.
Basically, this is correct.
You should see that:
    the lid has a u_star of 2.0, and the inner points immediately below it have a u_star of 0.2
    v_star should be still all zero
Note that we use ij indexing (not xy) which is why the field is sideways
"""
convert_vector_to_field(u_star, NX, NY)

array([[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 2. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 2. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 2. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 2. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 2. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 2. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 2. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 2. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 2. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 2. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 2. ]])

## Pressure Poisson Equation

\begin{equation} \tag{1.5}
\nabla^2 p^{n+1} = \frac{\rho}{\Delta t} \nabla \cdot \mathbf{u}^*
\end{equation}

Expanding:
\begin{equation} \nonumber{}
\frac{\partial^2 p^{n+1}}{\partial x^2} + \frac{\partial^2 p^{n+1}}{\partial y^2} = \frac{\rho}{\Delta t} \left( \frac{\partial u^*}{\partial x} + \frac{\partial v^*}{\partial y} \right)
\end{equation}

Discretizing:
\begin{equation} \nonumber{}

\end{equation}


In [ ]:
p_np1 = np.zeros(NX * NY)
A = np.zeros((NX * NY, NX * NY))
b = np.zeros(NX * NY)

# build linear system
for jdx in range(NY):
    for idx in range(NX):
        ij = jdx * NX + idx
        im1 = jdx * NY + idx - 1
        ip1 = jdx * NY + idx + 1
        jm1 = (jdx - 1) * NY + idx
        jp1 = (jdx + 1) * NY + idx
        # TODO

# solve (via closed form OLS here since the system is small)
# TODO

## Partially Implicit Step to Intermediate Velocity

\begin{equation} \notag{}
\frac{\mathbf{u}^* - \mathbf{u}^n}{\Delta t} = - (\mathbf{u}^n \cdot \nabla) \mathbf{u}^n + \nu \nabla^2 \mathbf{u}^*
\end{equation}

## Implicit Step to Intermediate Velocity

\begin{equation} \notag{}
\frac{\mathbf{u}^* - \mathbf{u}^n}{\Delta t} = - (\mathbf{u}^* \cdot \nabla) \mathbf{u}^* + \nu \nabla^2 \mathbf{u}^*
\end{equation}